In [62]:
'''Example taken from backtesting docs.'''

from backtesting import Backtest, Strategy
from backtesting.lib import crossover

# SMA is an example function provided by backtesting, any other strategy defined by user.
# GOOG and EUROSD are 2 example stocks provided by the module also, again any other must be provided by the user in the same format.
from backtesting.test import SMA, GOOG, EURUSD

'''Simple Moving Average Crossover to judge movement:
 - Uses moving averages with 2 different lags to judge the bulk movement of a stock;
     the lower sma lag responds faster than the higher, and crosses over it on its way up (down),
     the hope is that if there is only a temporary blip (shorter than the lag difference), then the eqaul trend will return before a crossover occurs.'''

# An example of strategic class structures - self is set to the inherited function 'Strategy', and the .
class SmaCross(Strategy):
    def init(self):
        price = self.data.Close
        # self.I is the indicator function:
        self.ma1 = self.I(SMA, price, 10)
        self.ma2 = self.I(SMA, price, 30)

    def next(self):
        # crossover calculates whether the cross-over is representing an increase or decrease:
        if crossover(self.ma1, self.ma2):
            self.buy()
        elif crossover(self.ma2, self.ma1):
            self.sell()


bt = Backtest(GOOG, SmaCross, commission=.002,
              exclusive_orders=True, cash=10_000)
stats = bt.run()
bt.plot()

Row(id='17482', ...)

In [60]:
print(stats)

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                     96.7412
Equity Final [$]                      44811.1
Equity Peak [$]                       68056.9
Return [%]                            348.111
Buy & Hold Return [%]                 703.458
Return (Ann.) [%]                     19.2393
Volatility (Ann.) [%]                   35.91
Sharpe Ratio                         0.535765
Sortino Ratio                        0.982666
Calmar Ratio                         0.390198
Max. Drawdown [%]                    -49.3065
Avg. Drawdown [%]                    -6.25496
Max. Drawdown Duration      584 days 00:00:00
Avg. Drawdown Duration       44 days 00:00:00
# Trades                                   66
Win Rate [%]                          43.9394
Best Trade [%]                        53.2894
Worst Trade [%]                      -18.6361
Avg. Trade [%]                    

In [77]:
## Backtesting optimisation of parameters - example again taken from docs.

%time

class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 10
    n2 = 20
    
    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()

bt = Backtest(GOOG, SmaCross, cash=10_000, commission=.002)
stats2 = bt.run()

# Optimise w.r.t. final equity, and add constraint that sma1 has lower lag than sma2:
stats_opt = bt.optimize(n1=range(5, 30, 5),
                    n2=range(10, 70, 5),
                    maximize='Equity Final [$]',
                    constraint=lambda param: param.n1 < param.n2)

# view results:
print(f'\nOptimal parameter statistics:\n')
print(stats_opt)

# We can look into stats['_strategy'] to access the Strategy instance and its optimal parameter values (10 and 15):
print(f'\nOptimal parameters:')
print(stats_opt._strategy)

Wall time: 0 ns



Optimal parameter statistics:

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                     99.0689
Equity Final [$]                       103949
Equity Peak [$]                        108328
Return [%]                            939.494
Buy & Hold Return [%]                 703.458
Return (Ann.) [%]                     31.6109
Volatility (Ann.) [%]                 44.7398
Sharpe Ratio                          0.70655
Sortino Ratio                         1.49096
Calmar Ratio                         0.718505
Max. Drawdown [%]                    -43.9954
Avg. Drawdown [%]                    -6.13885
Max. Drawdown Duration      690 days 00:00:00
Avg. Drawdown Duration       43 days 00:00:00
# Trades                                  153
Win Rate [%]                           51.634
Best Trade [%]                        61.5629
Worst Trade [%]                      -19.7783
Av

#### SMA Crossovers are a good predictor of the general trend of a stock value, but not fine-tuned or fast reacting enough to be an optimal strategy in of itself. Sensitivity is controlled by the difference in lag times.